In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Robot.sawyer import Sawyer
import rospy
import numpy as np

rospy.init_node('notebook_node')
robot = Sawyer()

##### Error Recovery

In [ ]:
!rostopic pub -1 /franka_ros_interface/franka_control/error_recovery/goal franka_msgs/ErrorRecoveryActionGoal "{}"

#### Record cartesian velocities

In [ ]:
cart_velocities = []
while True:
    cart_vel = robot.get_endpoint_velocity_in_base_frame()
    cart_velocities.append(cart_vel)
    rospy.sleep(0.5)

In [ ]:
len(cart_velocities)

In [ ]:
robot.robot.set_command_timeout(1.0)
for cart_vel in cart_velocities:
    robot.set_endpoint_velocity_in_base_frame(-cart_vel)
    rospy.sleep(0.5)
#robot.robot.exit_control_mode() 

#### Record joint velocities

In [ ]:
joint_velocities = []
while True:
    joint_velocity = robot.robot.velocities()
    joint_velocities.append(joint_velocity)
    rospy.sleep(0.5)

In [ ]:
len(joint_velocities)

In [ ]:
robot.robot.set_command_timeout(1.0)
for joint_velocity in joint_velocities:
    robot.robot.exec_velocity_cmd(joint_velocity)
    rospy.sleep(0.5)
#robot.robot.exit_control_mode() 

##### Stop velocity control in between with some velocity value

In [ ]:
robot.robot.set_command_timeout(1.0)
for i, joint_velocity in enumerate(joint_velocities):
    robot.robot.exec_velocity_cmd(joint_velocity)
    rospy.sleep(0.5)
    if i == 10: exit()
robot.robot.exit_control_mode() 

#### Record Efforts/torques and replay

In [ ]:
joint_efforts = []
while True:
    joint_effort = robot.robot.efforts()
    joint_efforts.append(joint_effort)
    rospy.sleep(0.5)

In [ ]:
len(joint_efforts)

In [ ]:
robot.robot.set_command_timeout(1.0)
for joint_effort in joint_efforts:
    robot.robot.exec_torque_cmd(joint_effort)
    rospy.sleep(0.5)
#robot.robot.exit_control_mode() 

#### Test Joint Position Control

In [ ]:

from Common import config
robot.move_to_joint_angles(config.ROBOT_INIT_JOINT_ANGLES)
